In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from tqdm import tqdm

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = "cpu"
seed_torch(2023)

In [20]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')
ALL_data = ECGHandle.filter_ages(ALL_data,18) 




            orginal   removed diagnose NaN
   nums      200082          199997       
              HTN             NHTN        
   nums       3273           196724       


            orginal      removed ID NaN   
   nums      199997          199995       
              HTN             NHTN        
   nums       3273           196722       


            orginal   filtered department 
   nums      199995          45571        
              HTN             NHTN        
   nums       3273           42298        


            orginal      filtered ages    
   nums      45571           43570        
              HTN             NHTN        
   nums       3220           40350        


In [21]:
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.correct_label(ALL_data)
ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ECGHandle.filter_diagnose(ALL_data,'起搏')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'房颤')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左束支传导阻滞')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左前分支阻滞')



            orginal            QC         
   nums      43570           15344        
              HTN             NHTN        
   nums       1477           13867        


     reset num:       10  
  ERR labels num:     27  
            orginal      correct label    
   nums      15344           15344        
              HTN             NHTN        
   nums       1513           13831        


   ERR ages num:      831 
            orginal       correct age     
   nums      15344           15344        
              HTN             NHTN        
   nums       1513           13831        


            orginal    remove diagnose起搏  
   nums      15344           15278        
              HTN             NHTN        
   nums       1494           13784        
           remove HTN     remove NHTN     
   nums        19              47         


            orginal    remove diagnose房颤  
   nums      15278           14747        
              HTN             NHTN        
   num

In [22]:
ALL_data[ALL_data['label'] == 0]['住院号'].unique().__len__()
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()

## 常规操作

In [ ]:
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()
seed_torch(2023)
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
# all_dataset = ECGHandle.ECG_Dataset(data_root,ALL_data_buffer,preprocess = True)
####################################################################随机选取test
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
test_dataset = ECGHandle.ECG_Dataset(data_root,test_df,preprocess = True)

In [ ]:
Models_path = '/workspace/data/Interpretable_HTN/model/20230322_030450/20230322_030450/BestF1_0.pt'
save_root = Models_path[:-3]+'/'    
layervalue_root = save_root+'/layervalue/'    
NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3), ] # type: ignore    
criterion = torch.nn.CrossEntropyLoss()    
testmodel = NET[0].to(DEVICE)    
testmodel.load_state_dict(torch.load(Models_path))    
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)      
y_true,y_pred,y_out,test_loss,test_acc = eval_model(test_dataloader,criterion,testmodel,DEVICE) # 验证模型

In [ ]:
np.array(y_out).shape

In [ ]:
test_df_save = test_df.copy()
test_df_save['predict HTN possibility'] = np.array(y_out)[:,1]
print(test_df_save.head)

In [ ]:
test_df_save[test_df_save['label']==1].__len__()

In [ ]:
test_df_save.to_csv('./test.csv',encoding='utf_8_sig')

In [ ]:

ALL_data.columns

In [ ]:
duplicated_HTN_df = ALL_data[ALL_data.duplicated(subset=['ID'], keep=False) & (ALL_data['label']==1)& ( ~ ALL_data['检查时间'].isnull()) ]

In [ ]:
duplicated_HTN_df.__len__()

In [ ]:
duplicated_HTN_df['检查时间'] = pd.to_datetime(duplicated_HTN_df['检查时间'])

In [ ]:
duplicated_HTN_df['date_diff'] = duplicated_HTN_df.groupby('ID')['检查时间'].apply(lambda x:abs( x.diff()).dt.total_seconds())

In [ ]:
duplicated_HTN_ID_list_buffer = list(duplicated_HTN_df[duplicated_HTN_df['date_diff']>(31536000/2)]['ID'])
print(duplicated_HTN_ID_list_buffer.__len__())

In [ ]:
duplicated_HTN_df= ALL_data[ALL_data['ID'].isin(duplicated_HTN_ID_list_buffer)].copy()
duplicated_HTN_dataset = ECGHandle.ECG_Dataset(data_root,duplicated_HTN_df,preprocess = True)

In [ ]:
Models_path = '/workspace/data/Interpretable_HTN/model/20230322_030450/20230322_030450/BestF1_0.pt'
save_root = Models_path[:-3]+'/'    
layervalue_root = save_root+'/layervalue/'    
NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3), ] # type: ignore    
criterion = torch.nn.CrossEntropyLoss()    
testmodel = NET[0].to(DEVICE)    
testmodel.load_state_dict(torch.load(Models_path))    
duplicated_HTN_dataloader = Data.DataLoader(dataset=duplicated_HTN_dataset, batch_size=1, shuffle=False)      
y_true,y_pred,y_out,test_loss,test_acc = eval_model(duplicated_HTN_dataloader,criterion,testmodel,DEVICE) # 验证模型

In [ ]:
duplicated_HTN_df['predict HTN possibility'] = np.array(y_out)[:,1]

In [ ]:
duplicated_HTN_df[duplicated_HTN_df['predict HTN possibility']<0.5]['ID'].unique().tolist()

In [ ]:
duplicated_HTN_df.to_csv('./duplicated_HTN_间隔半年以上.csv',encoding='utf_8_sig')

In [ ]:
jpg_path = './jpg/duplicated_HTN/'
for index in range(duplicated_HTN_dataset.__len__()):#test_dataset.__len__()
    info =duplicated_HTN_dataset.infos.iloc[index]
    file_name = info['ID']+'_'+info['ECGFilename']
    ID = info['ID']
    date = info['检查时间']
    age = info['age']
    label = info['label']
    ECG,labels = duplicated_HTN_dataset.__getitem__(index)
    ECG = ECG*5000 #恢复
    ecg_plot.plot(ECG*4.88/1000, sample_rate = 500, title = 'ID:'+str(ID)+' '+ 'label: '+ str(label) +' '+'Date: '+str(date)+' '+'age: '+str(age)  ,row_height= 10,show_grid=True,show_separate_line=True)
    ecg_plot.save_as_jpg(file_name,jpg_path)

## 测试集上分类错误的NHTN

In [ ]:
ERR_NHTN_df = test_df_save[ (test_df_save['label']==0) & (test_df_save['predict HTN possibility']>=0.5) ]
print(ERR_NHTN_df)

In [ ]:
ERR_NHTN_df.to_csv('./ERR_NHTN.csv',encoding='utf_8_sig')

## std & mean check

In [ ]:
FOLDS = 5
seed_torch(2023)
tv_df = tv_df.sample(frac=1).reset_index(drop=True) #打乱顺序

In [ ]:
for fold in range(FOLDS):
    print("Fold "+str(fold)+" of "+str(FOLDS) + ' :')
    tv_df_buffer = tv_df.copy()
    HTN_tv_df = tv_df[(tv_df['label']==1) ].copy()
    NHTN_tv_df = tv_df[(tv_df['label']==0) ].copy()
    HTN_ID_tv_list = HTN_tv_df['ID'].unique().tolist() #tvset中所有的HTN的ID号
    HTN_tv_size = HTN_tv_df['ID'].unique().__len__()
    HTN_validate_size = int(HTN_tv_size//FOLDS)
    validate_start_index = HTN_validate_size*fold #star index for validate
    validate_df,tarin_df = Pair_ID(tv_df_buffer,0.2,star_index=validate_start_index,Range_max=15,pair_num=1)
    validate_dataset = ECGHandle.ECG_Dataset(data_root,validate_df,preprocess = True)
    
    train_pair_df,_ = Pair_ID(tarin_df,1,star_index=0,Range_max=15,pair_num=1,shuffle=True)
    train_dataset = ECGHandle.ECG_Dataset(data_root,train_pair_df ,preprocess = True)
    for i in range(12):
        print('lead:' ,i)
        print('test:{}',test_dataset.datas[:,i,:].std(),test_dataset.datas[:,i,:].mean())
        print('train:{}',train_dataset.datas[:,i,:].std(),train_dataset.datas[:,i,:].mean())
        print('validat:{}',validate_dataset.datas[:,i,:].std(),validate_dataset.datas[:,i,:].mean())

In [ ]:
test_df[test_df['label'] == 0]['ID'].__len__()

In [ ]:
tv_df[tv_df['label'] == 0]['ID'].__len__()

## 查看训练集、测试集、验证集的年龄、性别分布是否有差别

In [ ]:
ALL_data.columns

In [ ]:
ALL_data.hist(column='age', by='gender')

In [ ]:
test_df.hist(column='age', by='gender')

In [ ]:
seed_torch(2023)
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
FOLDS = 5
seed_torch(2020)
tv_df = tv_df.sample(frac=1).reset_index(drop=True) #打乱顺序
for fold in range(FOLDS):
    print("Fold "+str(fold)+" of "+str(FOLDS) + ' :')
    tv_df_buffer = tv_df.copy()
    HTN_tv_df = tv_df[(tv_df['label']==1) ].copy()
    NHTN_tv_df = tv_df[(tv_df['label']==0) ].copy()
    HTN_ID_tv_list = HTN_tv_df['ID'].unique().tolist() #tvset中所有的HTN的ID号
    HTN_tv_size = HTN_tv_df['ID'].unique().__len__()
    HTN_validate_size = int(HTN_tv_size//FOLDS)
    validate_start_index = HTN_validate_size*fold #star index for validate
    validate_df,tarin_df = Pair_ID(tv_df_buffer,0.2,star_index=validate_start_index,Range_max=15,pair_num=1)
    
    train_pair_df,_ = Pair_ID(tarin_df,1,star_index=0,Range_max=15,pair_num=1,shuffle=True)
    validate_df.hist(column='age', by='gender')
    tarin_df.hist(column='age', by='gender')

#### 按照[18, 30, 40, 50, 60, 70, 110]年龄分组

In [34]:
# 按照指定的区间进行分组
bins = [18, 30, 40, 50, 60, 70, 110]
labels = ['18-30', '31-40', '41-50', '51-60', '61-70', '70-110']
ALL_data['agegroup'] = pd.cut(ALL_data['age'], bins=bins, labels=labels)

In [35]:
from sklearn.model_selection import train_test_split
# 按照agerange和gender进行分层抽样
train, test = train_test_split(ALL_data, test_size=0.2, stratify=ALL_data[['agegroup', 'gender']])

In [36]:
train.__len__()

11678

In [37]:
# 再次对训练集进行分层抽样，得到4份子集
subsets = []
for i in range(4):
    subset, train = train_test_split(train, test_size=0.2, stratify=train[['agegroup', 'gender']])
    subsets.append(subset)

## plot

In [ ]:
ALL_data.head()

In [ ]:
np.histogram(ALL_data[ALL_data['label']==1]['年龄'].to_numpy(),np.arange(20,100,10),density = True)

In [ ]:
np.histogram(ALL_data[ALL_data['label']==0]['年龄'].to_numpy(),np.arange(20,100,10),density = True)

In [ ]:
def filled_hist(ax, edges, values, bottoms=None, orientation='v',
                **kwargs):
    """
    Draw a histogram as a stepped patch.

    Parameters
    ----------
    ax : Axes
        The axes to plot to

    edges : array
        A length n+1 array giving the left edges of each bin and the
        right edge of the last bin.

    values : array
        A length n array of bin counts or values

    bottoms : float or array, optional
        A length n array of the bottom of the bars.  If None, zero is used.

    orientation : {'v', 'h'}
       Orientation of the histogram.  'v' (default) has
       the bars increasing in the positive y-direction.

    **kwargs
        Extra keyword arguments are passed through to `.fill_between`.

    Returns
    -------
    ret : PolyCollection
        Artist added to the Axes
    """
    print(orientation)
    if orientation not in 'hv':
        raise ValueError(f"orientation must be in {{'h', 'v'}} "
                         f"not {orientation}")

    kwargs.setdefault('step', 'post')
    kwargs.setdefault('alpha', 0.7)
    edges = np.asarray(edges)
    values = np.asarray(values)
    if len(edges) - 1 != len(values):
        raise ValueError(f'Must provide one more bin edge than value not: ',
                         len(edges), len(values))

    if bottoms is None:
        bottoms = 0
    bottoms = np.broadcast_to(bottoms, values.shape)

    values = np.append(values, values[-1])
    bottoms = np.append(bottoms, bottoms[-1])
    if orientation == 'h':
        return ax.fill_betweenx(edges, values, bottoms,
                                **kwargs)
    elif orientation == 'v':
        return ax.fill_between(edges, values, bottoms,
                               **kwargs)
    else:
        raise AssertionError("you should never be here")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,tight_layout=True)

ALL_data[(ALL_data['label']==1)&(ALL_data['性别']=='男')].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'HTN',ax=ax1)
ALL_data[(ALL_data['label']==1)&(ALL_data['性别']=='女')].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'HTN',ax=ax1)
ALL_data[(ALL_data['label']==0)&(ALL_data['性别']=='男')].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'NHTN',ax=ax1)
ALL_data[(ALL_data['label']==0)&(ALL_data['性别']=='女')].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'NHTN',ax=ax1)

ax2 = ALL_data[ALL_data['label']==1].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'HTN',ax=ax2)
ax2 = ALL_data[ALL_data['label']==0].年龄.plot.hist(stacked=True, bins=25, alpha=0.5,density=1,label = 'NHTN',ax=ax2)

In [ ]:
from scipy.stats import norm

fig, ax = plt.subplots(tight_layout=True)
n_bins = 50

data = ALL_data[ALL_data['label']==1]['年龄'].to_numpy()
# 拟合数据的正态分布参数: mu, std
mu, std = norm.fit (data)
ax.hist (data, n_bins, density=True,alpha = 0.5)
# 计算并绘制概率密度函数
xmin, xmax = plt.xlim ()
x = np.linspace (xmin, xmax, 100)
p = norm.pdf (x, mu, std)
ax.plot (x,p,'k',linewidth=1)

data = ALL_data[ALL_data['label']==0]['年龄'].to_numpy()
# 拟合数据的正态分布参数: mu, std
mu, std = norm.fit (data)
ax.hist (data, n_bins, density=True,alpha = 0.5)
# 计算并绘制概率密度函数
xmin, xmax = plt.xlim ()
x = np.linspace (xmin, xmax, 100)
p = norm.pdf (x, mu, std)
ax.plot (x,p,'k',linewidth=1)



plt.show()

In [ ]:
from scipy.stats import norm

fig, ax = plt.subplots(tight_layout=True)
n_bins = 50

data = ALL_data[ALL_data['label']==1]['年龄'].to_numpy()

y1,x1 = np.histogram(data,bins=30,density =True)
ax.bar(x1[1:],y1)

data = ALL_data[ALL_data['label']==0]['年龄'].to_numpy()
y2,x2 = np.histogram(data,bins=30,density =True)
ax.bar(x2[1:],y2,align='center')

# # 拟合数据的正态分布参数: mu, std
# mu, std = norm.fit (data)
# # 计算并绘制概率密度函数
# xmin, xmax = plt.xlim ()
# x = np.linspace (xmin, xmax, 100)
# p = norm.pdf (x, mu, std)
# ax.plot (x,p,'k',linewidth=1)





plt.show()

In [ ]:
jpg_path = './jpg/'
for index in range(test_dataset.__len__()):#test_dataset.__len__()
    info =test_dataset.infos.iloc[index]
    file_name = info['ECGFilename']
    ID = info['ID']
    date = info['检查时间']
    age = info['age']
    label = info['label']
    ECG,labels = test_dataset.__getitem__(index)
    ECG = ECG*5000 #恢复
    ecg_plot.plot(ECG*4.88/1000, sample_rate = 500, title = 'ID:'+str(ID)+' '+ 'label: '+ str(label) +' '+'Date: '+str(date)+' '+'age: '+str(age)  ,row_height= 10,show_grid=True,show_separate_line=True)
    ecg_plot.save_as_jpg(file_name,jpg_path)